# Participants
Multiple UAVs and Sensors (defined by variables `num_drones` and `num_sensors`) 

# Objectives
UAVs must collect data from all sensors at least once. UAVs are constantly attempting to collect 
information from the space around them, so data is automatically collected once they get close
enough to a sensor. When all sensors are visited an episode ends.

# Constraints
- Communication has limited range
- UAV speed is limited (and fixed at 10m/s)
- All participants must be within a square area of parametrized side.
- Episode length is limited

# Algorithm
The algorithm used to solve this problem was DDPG, adapted 
from https://docs.cleanrl.dev/rl-algorithms/ddpg/#ddpg_continuous_actionpy. A single RL agent is trained 
using information from all UAVs (Centralized training with parameter sharing).

The agents were integrated into GrADyS-SIM-Nextgen. Iterations happen every 0.5 seconds of simulation time.

## State
Agents (UAVs) know all agent's positions (including their own), they know their ID, they know the
positions of all sensors and they know if sensors were visited or not.

## Action
Agents chose a direction to travel

## Reward
- 1 to all agents if all sensor data has been collected
- -1 to a specific agent if it leaves the scenario area
- 0 otherwize

# Training
Episodes start with UAVs being placed randomly in a small area in the center of the scenario. Sensors are 
placed randomly outside of that small area. The simulation runs until the time limit. If an agent leaves
the scenario's area the simulation is terminated. If all sensors have been visited, the simulation is
terminated.

# Parameters
These are the parameters being studied:
- Number of drones [1, 2]
- Number of sensors [1, 2]
- Scenario size [50, 100] (size of the side of the scenario square)
- Training time [1_000_000, 10_000_000]


In [6]:
import torch
torch.device("cuda")

device(type='cuda')

In [6]:
!tar -czf runs.tar.gz runs

# Benchmark

In [1]:
import multiprocessing
import subprocess

parrallelism = 1

experiments = [
    ["python", "main.py", "--scenario_size=50", "--num-drones=1", "--num-sensors=1", f"--exp_name=bench_parallel_{parrallelism}cpu", "--max_episode_length=60", "--no-checkpoint-visual-evaluation", "--total-timesteps=100000"]
    for _ in range(parrallelism)
]

def run_experiment(experiment):
    print("Running experiment: ", experiment)
    subprocess.run(experiment)


if __name__ == "__main__":
    pool = multiprocessing.Pool(processes=parrallelism)
    pool.map(run_experiment, experiments)

Running experiment:  ['python', 'main.py', '--scenario_size=50', '--num-drones=1', '--num-sensors=1', '--exp_name=bench_parallel_1cpu', '--max_episode_length=60', '--no-checkpoint-visual-evaluation', '--total-timesteps=100000']


2024-05-09 22:52:22.119466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 22:52:22.277939: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/opt/tljh/user/lib/python3.9/site-packages/tyro/_fields.py:342: UserWarning: The field wandb_entity is annotated with type <class 'str'>, but the default value None has type <class 'NoneType'>. We'll try to handle this gracefully, but it may cause unexpected behavior.
  warnings.warn(


bench_parallel_1cpu - SPS: 3954
bench_parallel_1cpu - SPS: 3708
bench_parallel_1cpu - SPS: 3489
bench_parallel_1cpu - SPS: 3299
bench_parallel_1cpu - SPS: 3132
bench_parallel_1cpu - SPS: 2982
bench_parallel_1cpu - SPS: 2847
bench_parallel_1cpu - SPS: 2726
bench_parallel_1cpu - SPS: 2615
bench_parallel_1cpu - SPS: 2513
bench_parallel_1cpu - SPS: 2424
bench_parallel_1cpu - SPS: 2343
bench_parallel_1cpu - SPS: 2267
bench_parallel_1cpu - SPS: 2196
bench_parallel_1cpu - SPS: 2129
bench_parallel_1cpu - SPS: 2068
bench_parallel_1cpu - SPS: 2010
bench_parallel_1cpu - SPS: 1956
bench_parallel_1cpu - SPS: 1905
bench_parallel_1cpu - SPS: 1857
bench_parallel_1cpu - SPS: 1812
bench_parallel_1cpu - SPS: 1765
bench_parallel_1cpu - SPS: 1717
bench_parallel_1cpu - SPS: 1672
bench_parallel_1cpu - SPS: 1630
bench_parallel_1cpu - SPS: 1590
bench_parallel_1cpu - SPS: 1553
bench_parallel_1cpu - SPS: 1519
bench_parallel_1cpu - SPS: 1490
bench_parallel_1cpu - SPS: 1462
bench_parallel_1cpu - SPS: 1431
bench_pa

bench_parallel_1cpu - SPS: 301
bench_parallel_1cpu - SPS: 301
bench_parallel_1cpu - SPS: 301
bench_parallel_1cpu - SPS: 301
bench_parallel_1cpu - SPS: 301
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 300
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 299
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 298
bench_parallel_1cpu - SPS: 297
bench_parallel_1cpu - SPS: 297
bench_parallel_1cpu - SPS: 297
bench_parallel_1cpu - SPS: 297
bench_pa

## Results
| Parrallelism | Total SPS | Cuda? |
| ------------ | --------- | ----- |
| 10           | 940       | Yes   |
| 8            | 904       | Yes   |
| 1            | 280       | Yes   |

# Sensor number evaluation
Does the system performance scale well with higher sensor counts? Running the following scenarios to find out:

| Number of drones | Number of sensors | Sucess Rate |
| ---------------- | ----------------- | ----------- |
| 2                | 3                 |             |
| 2                | 4                 |             |
| 2                | 5                 |             |
| 2                | 6                 |             |
| 2                | 7                 |             |
| 2                | 8                 |             |
| 2                | 9                 |             |
| 2                | 10                |             |

## Simulações executadas:
```py
import multiprocessing
import subprocess
from dataclasses import dataclass

import tyro


@dataclass
class Args:
    concurrency: int = 3

experiments = [
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=3", "--exp_name=10mil_3-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=4", "--exp_name=10mil_4-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=5", "--exp_name=10mil_5-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=6", "--exp_name=10mil_6-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=7", "--exp_name=10mil_7-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=8", "--exp_name=10mil_8-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=9", "--exp_name=10mil_9-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
    ["python", "main.py", "--scenario_size=100", "--num-drones=2", "--num-sensors=10", "--exp_name=10mil_10-sensor-2-drone-100-size", "--max_episode_length=200", "--no-checkpoint-visual-evaluation", "--run-name=SensorIncrease", "--checkpoint-freq=1000000", "--total-timesteps=10000000"],
]

def run_experiment(experiment):
    print("Running experiment: ", experiment)
    subprocess.run(experiment)


if __name__ == "__main__":
    args = tyro.cli(Args)
    pool = multiprocessing.Pool(processes=args.concurrency)
    pool.map(run_experiment, experiments)
```